In [1]:
import googlemaps
from datetime import datetime
import time  # Import the time module to add a delay between API requests
import dotenv
import os
import pickle
import math
import re
import demoji
import emoji
import json

dotenv.load_dotenv(dotenv.find_dotenv())
gmaps = googlemaps.Client(key=os.getenv('GOOGLE_API_KEY'))

# Geocoding an address
geocode_result = gmaps.geocode('國立陽明交通大學光復校區')

In [2]:
def get_result(keyword: str):
    results = []
    loc = geocode_result[0]['geometry']['location']
    # query_result = gmaps.places_nearby(keyword="餐廳",location=loc, rank_by='distance', language='zh-TW')
    query_result = gmaps.places_nearby(keyword=keyword,location=loc, radius=1500, language='zh-TW')
    results.extend(query_result['results'])
    while query_result.get('next_page_token'):
        time.sleep(2)
        query_result = gmaps.places_nearby(page_token=query_result['next_page_token'])
        results.extend(query_result['results'])    
    
    return results

In [3]:
queries = ['餐廳', 
           '早餐店', 
           '早午餐', 
           '牛排', 
           '火鍋', 
           '宵夜', 
           '炸物', 
           '燒烤', 
           '麵食', 
           '飲料', 
           '甜點', 
           '冰品',
           '咖啡', 
           '異國料理', 
           '台式料理',
           '泰式料理',
           '美式料理',
           '素食',
           '速食',
           '豆花',
           '滷味',
           '日式料理',
           '壽司',
           '義式餐廳',
           '食物', 
           '簡餐', 
           '餐廳', 
           '外帶',
           '麻辣', 
           '內用', 
           '吃到飽',
           '北方麵食',
           '熱炒',
           '拉麵',
           'pizza',
           '炸雞',
           '外送',
           '港式料理',
           '鴨肉',
           '雞肉',
           '羊肉',
           '鵝肉',
           '湯包',
           '鐵板燒',
           '便當',
           '焗烤',
           '炒飯',
           '小吃',
           '串燒',
           '牛肉']
results = []
place_ids = []

for query in queries:
    result = get_result(query)
    for r in result:   
        if r['place_id'] in place_ids:
            # add keyword to result
            results[place_ids.index(r['place_id'])]['keyword'].append(query)
            continue
        r['keyword'] = [query, ]
        results.append(r)
        place_ids.append(r['place_id'])

In [4]:
# use pickle to save the results
with open('raw_results.pickle', 'wb') as f:
    pickle.dump(results, f)

In [5]:
len(results)

545

In [6]:
current_loc = geocode_result[0]['geometry']['location']

# measure distance by lat and lng
def measure(loc1, loc2):  # generally used geo measurement function
    R = 6378.137  # Radius of earth in KM
    lat1, lon1 = loc1['lat'], loc1['lng']
    lat2, lon2 = loc2['lat'], loc2['lng']
    dLat = lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon = lon2 * math.pi / 180 - lon1 * math.pi / 180
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + math.cos(lat1 * math.pi / 180) * math.cos(
        lat2 * math.pi / 180) * math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c
    return d # in km

# replace(if possible) and remove emoji
def replace_emoji(text):    
    dem = demoji.findall(text)
    for item in dem.keys():
        text = text.replace(item, f':{dem[item]}:')

    return emoji.replace_emoji(text, replace='')

In [7]:
remain_keys = ['name', 'rating', 'keyword'] #, 'vicinity', 'place_id', 'geometry'] 

for result in results:
    # place_detail = gmaps.place(result['place_id'], fields=['reviews', 'serves_breakfast', 'serves_brunch', 'serves_dinner', 'serves_lunch'], language='zh-TW')
    place_detail = gmaps.place(result['place_id'], fields=['reviews'], language='zh-TW')
    dest_loc = result['geometry']['location']
    
    for key in list(result.keys()):
        if key not in remain_keys: del result[key]
        
    result['distance'] = measure(current_loc, dest_loc)
    reviews = [review['text'] for review in place_detail['result']['reviews']] if place_detail['result'].get('reviews', False) else []
    reviews = [replace_emoji(review) for review in reviews] 
    
    result['reviews'] = '\n\n\n'.join(reviews) # reviews
    result['keyword'] = ', '.join(result['keyword']) # keyword
    
    print(result['name'])

肥仔龍無煙鐵板燒
經濟部專業人員研究中心餐廳
橄欖樹廚房
野叔主廚私房料理
新天地美食館
禧樂 丼飯、刺身、燒物
儷舍坊
YOUKO 串燒き
露米亞歐法餐廳
麥當勞-新竹光復餐廳
白鬍子牛排清大店
山威食堂
Westwood 現代美式餐廳
台積館品漾餐廳
托斯卡尼尼義大利餐廳-竹科店
LALA Kitchen 交大店
麥當勞-新竹清大餐廳
十六區和風料理
甜夜燒肉 Honey Night
街巷Coffee&Restaurant
湘月食堂
英美食坊
麥當勞-新竹光復二餐廳
漢神鐵板燒／新竹鐵板燒／新竹美食／新竹餐廳／新竹必吃／光復路必吃／光復路美食
HOUSE KITCHEN 好事廚房｜LIFE KITCHEN 生活倉廚｜法式藜麥餐盒｜高級會議餐盒｜舒肥健康餐｜新竹商務便當｜輕食餐廳 ｜Takeout Restaurant｜Health Food Restaurant
泰式緬甸風味館
台灣光罩員工餐廳
RINGOAL車庫餐廳
水木餐廳
樂樂ENJOY PASTA
和選旅-硯宴軒
山口刺身丼飯專賣店-清大店/新竹美食/清大夜市/生魚片/平價丼飯/日本料理
禾園茶館
廚種 新竹學府店
瘋丼-新竹馬偕店-新竹外送美食|外送便當|日式丼飯|商務便當|推薦丼飯|必吃餐廳
肯德基 新竹光復餐廳
明昌越式餐館-新竹美食-東區餐廳-河粉-越南料理-特色料理
肯德基 新竹清大餐廳
沐啡
哩賀巴西小館
清華水漾餐廳
謐‧義 La soffritto
原味燉品屋-建中店
國立陽明交通大學第二餐廳
璽子牛肉麵
車庫町日式咖哩，丼飯
十上食堂
新竹哈奇客韓式炸雞
饗食堂 麵飯飲料
阿彤北和牛 海鮮 燒肉 定食 新竹總店
國立陽明交通大學第一餐廳
好咖啡手工窯烤披薩
安妞韓國烤肉食堂-新竹竹科店
老農家越式小館
也果蔬食 竹科店
馬六甲馬來西亞風味餐廳清大店
初乎義料 chū hū Pasta
佬法室早午餐 Old Brunch
清大雙喜餛飩、老虎麵
La stella義式人文餐廳
來來豆漿店
永和豆漿大王
Friend’s早餐店
美之晨早餐屋
美天美早餐漢堡店
瑞斯飯糰清大店
早餐屋
No. 19活力廚房
美而佳早餐店
家鄉味早餐
g'day早餐店
8+8 早餐坊（高翠路175號）
幸福屋早餐坊
品香早餐
無名早餐店
無名早餐店 饅頭肉排蛋
安迪早餐屋(安迪早午餐)
拉亞漢堡 新竹建新
早安漢

In [8]:
results = [result for result in results if len(result['reviews']) > 120]

In [9]:
print(len(results))

514


In [10]:
# save processed result
with open('proc_results.pickle', 'wb') as f:
    pickle.dump(results, f)
    
# save as json file
with open('proc_results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

In [11]:
results

[{'name': '肥仔龍無煙鐵板燒',
  'rating': 3.2,
  'keyword': '餐廳, 餐廳, 鐵板燒',
  'distance': 1.1925729708669994,
  'reviews': '今年來清大讀書，接近學期末才來吃，只能說我覺得可以接受，並不會認為像某些評論所批評的那麼誇張。\n\n首先，價格真的是非常親民，最便宜的套餐直接標榜學生餐，除了會附贈的白飯、濃湯、飲料等，一個肉加一個魚、兩樣青菜，這樣只要$120，再加上一顆蛋也只要$20，真的是沒有什麼好挑剔的了，一定是學生族群的月底救星吧！\n\n再者，不知道是不是每次都如此，用餐的當天外送非常的多，所以出餐速度會稍微較慢，像入店後超過半小時青菜跟肉菜陸續上桌，不適合趕時間的人來用餐。\n\n第三，大概知道大家在擔心的衛生問題，但我自己覺得還好，除了抽屜有點黏（自然現象），吃完身上有一點點臭之外，剩下都可以接受。\n\n最後，店員其實只是因為太過忙碌有點脾氣不好，大家可以不用那麼計較，畢竟用餐是真的很便宜，當天負責炒肉的先生，還有跟顧客們說聲抱歉，我覺得已經仁至義盡了，大家就別多苛求，真的不行就吃其他間吧~~\n\n\n2023/11\n很難想到現在還有120元可以吃到的鐵板燒\n店位在清華夜市裡\n旁邊有停車場（只能悠遊卡付款）\n店裡有兩大區料理平台\n看起來舊舊的\n平日中午1點左右到\n大概只坐了1/3\n\n點了學生套餐-牛肉（120元）包含豆芽菜、高麗菜、鯛魚片、牛肉片（不是學生也可以點學生套餐，另外有豬肉、羊肉可以選）\n點完餐後店家會送上白飯、濃湯、飲料杯\n濃湯、飲料喝到飽\n餐點味道都算正常\n豆芽菜、高麗菜都很脆口\n鯛魚片略乾\n牛肉片是一般黑胡椒口味\n每個人的餐點都集中放在一個盤子裡\n師傅料理完就直接堆疊起來\n會有味道互相干擾的情形\n個人比較不喜歡\n\n現場看不管是加辣、不吃豆芽菜換高麗菜等，都能配合客製調整，服務真的不錯\n\n整體來說CP值超高\n靠近學校的學生美食\n有順路經過值得一試\n\n\n超好吃～很南部人口味，有個師傅很幽默:grinning face:下次還會再光顧…台南人大推\n\n\n便宜\n吃的很飽\n只是有點油膩\n畢竟要服務的人太多了\n一個學生餐，可能把附近餐飲打趴\n值得來試試CP值超高的本土鐵板燒\n\